# DS-GA 1001 Project
## Training Set and Feature Engineering (Part 1)

This script creates training sample for Project Part 1 (Please refer to readme for details)
***

## 1. Load Data

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics 
%matplotlib inline
from sklearn.model_selection import train_test_split

In [13]:
Data = pd.read_csv("mortgage.csv")
Data.head()

,id,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,...,REtype_SF_orig_time,investor_orig_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time,payoff_time,status_time
0,1,25,-7,25,113,41303.42,24.498336,9.2,226.29,2.899137,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
1,1,26,-7,25,113,41061.95,24.483867,9.2,225.10,2.151365,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
2,1,27,-7,25,113,40804.42,24.626795,9.2,222.39,2.361722,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
3,1,28,-7,25,113,40483.89,24.735883,9.2,219.67,1.229172,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
4,1,29,-7,25,113,40367.06,24.925476,9.2,217.37,1.692969,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0


**Drop Duplicate Values**

In [14]:
Data.drop_duplicates(keep = False, inplace = True)

## 2. Fill NA Values

There are two variables containing missing values: LTV_time and Interest_Rate_Orig_Time. <br>
Since both variables are highly correlated with other variables, we fill in NA from an estimation of polynomial regression from other existing variables, with "LinearRegression" in sklearn package. We will not adopt all variables in feature engineering in later steps, so our filling NA method won't cause multicolinearity issue in the main model training. 

### 2.1 FillNA for LTV_time

In [3]:
Data['LTV_time'][Data['LTV_time']==0] = None

ltv_sample = Data[~Data['LTV_time'].isnull()]
corr = ltv_sample.corr()
features = ltv_sample.columns[abs(corr.loc['LTV_time',:]) > 0.3]
ltv_sample = ltv_sample[features]
ltv_result_x = Data[features][Data['LTV_time'].isnull()].drop(['LTV_time'], axis = 1)


/Users/yuweiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [4]:
train, test = train_test_split(ltv_sample, test_size=0.2)
X_train, X_test = train.drop(['LTV_time'], axis = 1), test.drop(['LTV_time'], axis = 1)
y_train, y_test = train['LTV_time'], test['LTV_time']

In [5]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# score = {}
# for degree in [1,2,3]:
#     model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
#     model.fit(X_train,y_train)
#     score[degree] = model.score(X_test, y_test)
# display(score)

In [6]:
from sklearn.metrics import mean_squared_error, r2_score
model = make_pipeline(PolynomialFeatures(3), LinearRegression())
model.fit(X_train,y_train)
# Make predictions using the testing set
y_pred = model.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 64.62
Variance score: 0.90


In [7]:
ltv_result_y = model.predict(ltv_result_x)
Data.loc[Data['LTV_time'].isnull(), 'LTV_time'] = ltv_result_y

### 2.2 FillNA for Interest_Rate_orig_time

In [8]:
Data['Interest_Rate_orig_time'][Data['Interest_Rate_orig_time']==0] = None

/Users/yuweiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
temp = Data['id'][Data['Interest_Rate_orig_time'].isnull()].value_counts()
fill_val = {}
for i in temp.index:
  fill_val[Data[Data['id']==i].index[0]] = Data[Data['id'] == i]['interest_rate_time'].iloc[0]

In [10]:
Data['Interest_Rate_orig_time'] = Data['Interest_Rate_orig_time'].fillna(fill_val)
Data.fillna(method = 'ffill', inplace=True)

## 3. Feature Engineering Functions (for time-series features)

### 3.1 Get a complete monthly payment history for certain borrower for all available periods

Function: **MonthlyPayment(ID)**
- Input: ID
- Return: a list containing monthly payment for all available months
- Method: detect starting and ending (first and last) available observation time, calculate difference between two remaining balances and store into list. 

In [125]:
def MonthlyPayment(ID):
    Payment = []
    
    timeStart = Data.groupby(['id'])['time'].min()
    timeEnd = Data.groupby(['id'])['time'].max()
    timeStart = timeStart.to_frame()
    timeEnd = timeEnd.to_frame()
    
    for time in range(timeStart.loc[ID].values[0], timeEnd.loc[ID].values[0]):
        condition1 = (time+1 <= timeEnd.loc[ID].values[0])
        condition2 = (time in set(Data[Data['id']==ID]['time']))
        condition3 = (time+1 in set(Data[Data['id']==ID]['time']))
        
        if condition1 and condition2 and condition3:
            Balance1 = Data[Data['id']==ID][Data['time']==time]['balance_time'].values[0]
            Balance2 = Data[Data['id']==ID][Data['time']==(time+1)]['balance_time'].values[0]
            monthlyPay = Balance1 - Balance2
            Payment = Payment + [monthlyPay]
        else:
            continue
    return Payment

### 3.2 Mean, Std, Min, Max for total payment history or up to given time

In [139]:
def meanMonthlyPayment(ID):
    if len(MonthlyPaymentNew(ID))>=1:
        return statistics.mean(MonthlyPaymentNew(ID))
    else:
        return 0

def stdMonthlyPayment(ID):
    if len(MonthlyPaymentNew(ID))>= 2:
        return statistics.stdev(MonthlyPaymentNew(ID))
    else:
        return 0

def maxMonthlyPayment(ID):
    if len(MonthlyPaymentNew(ID))>=1:
        return max(MonthlyPaymentNew(ID))
    else:
        return 0

def minMonthlyPayment(ID):
    if len(MonthlyPaymentNew(ID))>=1:
        return min(MonthlyPaymentNew(ID))
    else:
        return 0

A combined function for all features above: **CombinedMonthlyPayment(ID)**
- Input: Borrower ID
- Return: A list of following values:
    - avg: average payment per month
    - maxi: maximum payment
    - mini: minimum payment
    - std: standard deviation of peyment
    - record0: number of 0 amount payment throughout history
    - recordLow: number of payments lower than mean - 1*std in history
- Method: 
    - Call MonthlyPayment to get a series for history
    - Calculate avg, maxi, mini, std from built-in functions
    - record0 and recordLow are counted from for loop

In [22]:
# This is a combined function, to improve efficiency
def CombinedMonthlyPayment(ID):
    sr = MonthlyPayment(ID)
    
    if len(sr)>=1:
        avg = statistics.mean(sr)
        maxi = max(sr)
        mini = min(sr)
        if len(sr)>=2:
            std = statistics.stdev(sr)
        else:
            std = 0
    else:
        avg = 0
        maxi = 0
        mini = 0
        std = 0
    
    record0 = 0
    recordLow = 0
    for i in range(len(sr)):
        if sr[i] <= 0:
            record0 += 1
        if sr[i] <= avg-std:
            recordLow += 1
    
    return (avg, std, maxi, mini, record0, recordLow)

## 4. Create training set (by borrower)

**List of features**
- mat_time: maturity time for loan (as original)
- balance_orig: initial balance, from balance_orig_time
- balance_last: remaining balance before default/payoff
- LTV_avg: average LTV_time
- LTV_orig: same as LTV_orig_time
- interest_avg: average interest_rate_time
- interest_orig: same as Interest_Rate_orig_time
- hpi_avg: average hpi_time
- hpi_orig: same as hpi_orig_time
- gdp_avg: average gdp_time
- uer_avg: average uer_time
- REtype_CO, REtype_PU, REtype_SF: as original
- investor, FICO: same as _orig_time

**Created features**
- payment_hist: total length of history, time.last - orig_time
- avg_payment: average amount of payment every month
- std_payment: std of payment every month
- max_payment: max of payment every month
- min_payment: min of payment every month
- count_zero_payment: number of 0 payments
- count_low_payment: number of payments < mean - 1*std

### 4.1 Original Features

In [33]:
# This part creates a dataframe containing all features listed above except for "Created features"

mat_time = Data.groupby(['id'])['mat_time'].first()
orig = Data.groupby(['id'])['investor_orig_time', 'balance_orig_time', 'FICO_orig_time',
       'LTV_orig_time', 'Interest_Rate_orig_time', 'hpi_orig_time',].first()
orig.rename(columns={"investor_orig_time": "investor", "balance_orig_time": "balance_orig", "FICO_orig_time": "FICO"
                    , "LTV_orig_time":"LTV_orig", "Interest_Rate_orig_time": "interest_orig", 
                    "hpi_orig_time": "hpi_orig"}, inplace=True)

balance_last = Data.groupby(['id'])[['balance_time']].last()
balance_last.rename(columns={"balance_time": "balance_last"}, inplace=True)

means = Data.groupby(['id'])['LTV_time', 'interest_rate_time','hpi_time', 
                            'gdp_time','uer_time'].mean()
balance_last.rename(columns={"LTV_time": "LTV_avg", 'interest_rate_time': 'interest_avg', 
                            'hpi_time':'hpi_avg', 'gdp_time': 'gdp_avg', 'uer_time': 'uer_avg'}, inplace=True)

REtype = Data.groupby(['id'])['REtype_CO_orig_time', 'REtype_PU_orig_time', 'REtype_SF_orig_time'].first()
REtype.rename(columns={"REtype_CO_orig_time": "REtype_CO", "REtype_PU_orig_time":'REtype_PU',
                      "REtype_SF_orig_time": 'REtype_SF'}, inplace=True)

target = Data.groupby(['id'])['default_time', 'payoff_time','status_time'].max()

Part1Data = pd.DataFrame(mat_time)
Part1Data = Part1Data.join(orig).join(balance_last).join(means).join(REtype).join(target)
Part1Data.head()

,mat_time,investor,balance_orig,FICO,LTV_orig,interest_orig,hpi_orig,balance_last,LTV_time,interest_rate_time,hpi_time,gdp_time,uer_time,REtype_CO,REtype_PU,REtype_SF,default_time,payoff_time,status_time
id,,,,,,,,,,,,,,,,,,,
1,113,0,45000.0,715,69.4,9.200,87.03,29087.21,26.319186,9.200000,176.657917,0.879284,7.241667,0,0,1,1,0,1
2,138,0,107200.0,558,80.0,7.680,186.91,105654.77,65.350597,7.680000,225.695000,2.525251,4.700000,0,0,1,0,1,2
3,114,0,48600.0,680,83.6,8.750,89.58,44378.60,30.953371,11.025000,222.164000,2.066873,4.580000,0,0,1,0,1,2
4,119,0,63750.0,587,81.8,10.500,97.99,52686.35,41.626867,10.107143,176.642286,1.264322,7.162857,0,0,1,0,0,0
5,138,0,52800.0,527,80.0,9.155,186.91,52100.71,65.821242,9.155000,224.593333,2.470741,4.600000,0,0,1,0,1,2


### 4.2 Created Features

In [35]:
# Payment History Length for each borrower
payment_hist = Data.groupby(['id'])['time'].last() - Data.groupby(['id'])['orig_time'].first()
Part1Data['payment_hist'] = payment_hist

**Fill time series features by CombinedMonthlyPayment(ID), for loop over each ID in data set**
(The output only shows for one dataset. The running process for other data sets have been removed)

In [ ]:
# Everything else in the list
avg_payment = []
std_payment = []
max_payment = []
min_payment = []
count_zero_payment = []
count_low_payment = []
IDcount = 0

for ID in range(1,50000+1):
    v1, v2, v3, v4, v5, v6 = CombinedMonthlyPayment(ID)
    avg_payment.append(v1)
    std_payment.append(v2)
    max_payment.append(v3)
    min_payment.append(v4)
    count_zero_payment.append(v5)
    count_low_payment.append(v6)
    IDcount +=1
    
    # The print line is only for helping track process
    print(IDcount)
    
# Running output was removed due to space limit

### 4.3 Put calculated values into new columns

In [147]:
Part1Data['avg_payment'] = avg_payment
Part1Data['std_payment'] = std_payment
Part1Data['max_payment'] = max_payment
Part1Data['min_payment'] = min_payment
Part1Data['count_zero_payment'] = count_zero_payment
Part1Data['count_low_payment'] = count_low_payment

In [148]:
Part1Data.head()

,mat_time,investor,balance_orig,FICO,LTV_orig,interest_orig,hpi_orig,balance_last,LTV_time,interest_rate_time,...,default_time,payoff_time,status_time,payment_hist,avg_payment,std_payment,max_payment,min_payment,count_zero_payment,count_low_payment
id,,,,,,,,,,,,,,,,,,,,,
1,113,0,45000.0,715,69.4,9.200,87.03,29087.21,26.319186,9.200000,...,1,0,1,55,531.139565,787.563229,3841.63,0.00,3,0
2,138,0,107200.0,558,80.0,7.680,186.91,105654.77,65.350597,7.680000,...,0,1,2,8,171.610000,0.000000,171.61,171.61,0,1
3,114,0,48600.0,680,83.6,8.750,89.58,44378.60,30.953371,11.025000,...,0,1,2,35,121.895000,4.120142,126.17,116.93,0,1
4,119,0,63750.0,587,81.8,10.500,97.99,52686.35,41.626867,10.107143,...,0,0,0,62,235.307879,66.681358,361.85,148.68,0,8
5,138,0,52800.0,527,80.0,9.155,186.91,52100.71,65.821242,9.155000,...,0,1,2,9,97.180000,1.569777,98.29,96.07,0,0


In [149]:
Part1Data.to_csv('Part1Data.csv')